In [ ]:
import pybedtools
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import sys

In [5]:
def get_list_filter_ids():
    File = open("Filtered_de_novo_transcripts_homologs.csv", "r") #to use the same data as in the paper and show how its different now
    File.readline()
    Accepted = []
    for line in File:
        l = line.split(",")
        if "Homolog" in l[3]:
            id = "NonExpressedHomolog_" + l[1] + "_"+ l[2] + "_" + l[1].split("::")[1]
        else:
            id = l[1].split("::")[0]+ "_"+ l[2] 
        Accepted.append(id)
    return Accepted

In [ ]:
def run_pybedtools_intersect(seqFile, teFile, pop, Type, TEtool):
    #Run pybedtools intersect and output a dictionary
    a = pybedtools.example_bedtool(seqFile)
    b = pybedtools.example_bedtool(teFile)
    c = a.intersect(b, wao = True, s = True)
    Out = {}
    Accepted = get_list_filter_ids()
    for elem in c:
        if elem[3] not in Accepted and elem[3]+ "_" + pop not in Accepted:
            continue
        if elem[-1] == "0":
            TEclass = "NoTE"
        elif TEtool == "EarlGrey":
            TEclass = elem[9].split("/")[0]
        elif TEtool == "FasTE":
            TEclass = elem[12].split("__")[1].split("_")[0]
        elif TEtool == "MCHelper" or TEtool == "MCHelper_EDTA":
            TEclass = elem[12] 
        elif TEtool == "RepeatModeler":
            TEclass =elem[12]
        if TEclass in ["LTR", "LINE", "ClassI", "CLASSI"]:
            TEclass = "RNA"
        elif TEclass != "NoTE" and TEclass != "unknown" and TEclass != "Unknown":
            TEclass = "DNA"
        elif TEclass == "unknown":
            TEclass = "Unknown"
        if elem[3] + "_" + pop + "_" + Type in Out:
            Out[elem[3] + "_" + pop + "_" + Type].append(TEclass)
            Out[elem[3] + "_" + pop + "_" + Type] = sorted(list(set(Out[elem[3] + "_" + pop + "_" + Type])))
        else:
            Out[elem[3] + "_" + pop + "_" + Type] = [TEclass]

    return Out


In [7]:
def run_for_all(TEtool):
    #Create the output file for all necessary input seqs
    Out = open(f"Fig1C_Dataframe_{TEtool}.csv", "w")
    Out.write("ID,Type,Group\n")
    TranslatePopDict = {"AK5":"FI", "DK5":"DK", "GI5":"ES", "SW5":"SE", "UM":"UA", "YE":"TR", "Zamb":"ZI"}
    for pop in ["AK5", "DK5", "GI5", "SW5", "UM", "YE", "Zamb"]:

        #DeNovo
        up = run_pybedtools_intersect(f"/global/group/research/m_lebh01/TEpaperCorrection/Bedfiles/Upstream/{pop}UpstreamTransformed.bed", f"/global/group/research/m_lebh01/TEpaperCorrection/TEannotationBed/{TEtool}/{TranslatePopDict[pop]}TEonly.bed", pop, "Upstream", TEtool)
        dn = run_pybedtools_intersect(f"/global/group/research/m_lebh01/TEpaperCorrection/Bedfiles/Downstream/{pop}EndTransformed.bed", f"/global/group/research/m_lebh01/TEpaperCorrection/TEannotationBed/{TEtool}/{TranslatePopDict[pop]}TEonly.bed", pop, "Downstream", TEtool)
        tr = run_pybedtools_intersect(f"/global/group/research/m_lebh01/TEpaperCorrection/Bedfiles/Transcript/{TranslatePopDict[pop]}Transformed.bed", f"/global/group/research/m_lebh01/TEpaperCorrection/TEannotationBed/{TEtool}/{TranslatePopDict[pop]}TEonly.bed", pop, "Transcript", TEtool)
        for elem in up:
            Out.write(f"{elem},{'_'.join(up[elem])},DeNovo\n")
        for elem in tr:
            Out.write(f"{elem},{'_'.join(tr[elem])},DeNovo\n")
        for elem in dn:
            Out.write(f"{elem},{'_'.join(dn[elem])},DeNovo\n")

        #Homolog
        up = run_pybedtools_intersect(f"/global/group/research/m_lebh01/TEpaperCorrection/Bedfiles/Homologs/{pop}_homologs_upstream.bed", f"/global/group/research/m_lebh01/TEpaperCorrection/TEannotationBed/{TEtool}/{TranslatePopDict[pop]}TEonly.bed", pop, "Upstream", TEtool)
        dn = run_pybedtools_intersect(f"/global/group/research/m_lebh01/TEpaperCorrection/Bedfiles/Homologs/{pop}_homolog_downstream.bed", f"/global/group/research/m_lebh01/TEpaperCorrection/TEannotationBed/{TEtool}/{TranslatePopDict[pop]}TEonly.bed", pop, "Downstream", TEtool)
        tr = run_pybedtools_intersect(f"/global/group/research/m_lebh01/TEpaperCorrection/Bedfiles/Homologs/NE_homologues{pop}2.bed", f"/global/group/research/m_lebh01/TEpaperCorrection/TEannotationBed/{TEtool}/{TranslatePopDict[pop]}TEonly.bed", pop, "Transcript", TEtool)
        for elem in up:
            Out.write(f"{elem},{'_'.join(up[elem])},Homolog\n")
        for elem in tr:
            Out.write(f"{elem},{'_'.join(tr[elem])},Homolog\n")
        for elem in dn:
            Out.write(f"{elem},{'_'.join(dn[elem])},Homolog\n")

In [19]:
def plot(TEtool):
    #Implement Bertrands R plot similarly in python
    plt.clf()
    df = pd.read_csv(f"Fig1C_Dataframe_{TEtool}.csv")
    df = df[df["Type"]!= "NoTE"]
    df["TEClass"] = df["Type"]
    df.loc[df["Type"].str.split("_").str.len() > 1, "TEClass"] = "Mix"
    ax = sns.histplot(data = df, x = "Group", hue = "TEClass", hue_order= ["DNA", "Mix", "RNA", "Unknown"],multiple="fill", palette=["#CFF0CC", "#43392F", "#BFAEBB", "grey"])
    ax.set(title = f"TE tool = {TEtool}")
    plt.ylabel("proportion of seqs")
    plt.savefig(f"Fig3C_{TEtool}.jpg",  bbox_inches="tight")

In [ ]:
def main(TEtool):
    #Run for all
    run_for_all(TEtool)
    plot(TEtool)